In [35]:
from PIL import Image
from PIL import ImageDraw
import timeit
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from skimage.feature import greycomatrix, greycoprops
import matplotlib.image as mpimg 
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import random

In [69]:
map_8bit_to_3bit = [i // 32 for i in range(256)]
model = {}
model_test = {}
neighbor = 3
time_modelling_start = timeit.default_timer()
model["cap"] = []
model["tulis"] = []
model_test["cap"] = []
model_test["tulis"] = []

In [60]:
#Pre Processing
def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img)
    
def get_img_colors(img, sampling_count):
    img_width, img_height = get_img_size(img)
    
    sample_width = int(img_width / sampling_count)
    sample_height = int(img_height / sampling_count)
    
    half_sample_width = int(sample_width/2)
    half_sample_height = int(sample_height/2)

    img_colors = []

    width_constraint = img_width - (2 * sample_width)
    height_constraint = img_height - (2 * sample_height)
   
    row = 0
    for point_x in range(half_sample_width, img_width, sample_width):
        if(row == sampling_count):
            break
        column = 0
        for point_y in range(half_sample_height, img_height, sample_height):
            if(column == sampling_count):
                break
            img_colors.append(img.getpixel((point_x, point_y)))
                
            column += 1
                
        row += 1
            
    return img_colors

def construct_img(img_colors, img_dimension, sampling_count):
    img_new = Image.new('RGB', (img_dimension), (255, 255, 255))
    draw = ImageDraw.Draw(img_new)
    
    img_new_width = img_dimension[0]
    img_new_height = img_dimension[1]

    sample_width = int(img_new_width/sampling_count)
    sample_height = int(img_new_height/sampling_count)
    loop_count = 0

    img_colors_len = len(img_colors)

    for w in range(0, img_new_width, sample_width):
        for h in range(0, img_new_height, sample_height):
            if(loop_count == img_colors_len):
                break
            
            current_color = (img_colors[loop_count], img_colors[loop_count], img_colors[loop_count])
            draw.rectangle((w, h, w+sample_width, h+sample_height), fill=current_color)
            loop_count += 1
    
    return img_new

def get_3bit_colors(img_colors):
    loop_count = 0
    
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = [] 
    loop_count = 0
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix


In [61]:
#Features Extraction
def feature_extract(img, sampling_count, class_name):
    glcm_component=[]
    img_colors = get_img_colors(img, sampling_count)
    img_3bit_colors = get_3bit_colors(img_colors)
    img_matrix = get_img_matrix(img_3bit_colors)
    glcm_matrix=greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    component = ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity']
    for x in component:
        glcm_component.append(greycoprops(glcm_matrix, x)[0][0])
    glcm_component.append(class_name)
    return(glcm_component)

In [62]:
#Modelling
def get_class_names(training_folder_path):
    return os.listdir(training_folder_path)

def make_a_model(class_name, features, learning_rate):
    random_splitter = random.uniform(0, 1)
    if(random_splitter <= learning_rate):
        model[class_name].append(features)
    else:
        model_test[class_name].append(features)
        
def to_dataframe(model):
    models = pd.DataFrame(model['cap'] + model['tulis'], columns=['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity', 'class'])
    return models

In [63]:
#Classification
def euclidian_distance(img_validate, img_model):
    distance = 0.0
    for i in range(len(img_validate)-1):
        distance += (img_validate[i] - img_model[i])**2
    return sqrt(distance)

def knn(img, sampling_count, neighbor):
    min_distance = []
    class_min_distance = []
    k = 0
    img_validate = feature_extract(img, sampling_count)
    distance = 0.0
    for name_class in model:
        for img_train in model[name_class]:
            distance = euclidian_distance(img_validate, img_train)
            
            if(k<neighbor):
                min_distance.append(distance)
                class_min_distance.append(name_class)
                k+=1
            else:
                for i in range (k):
                    if(distance<min_distance[i]):
                        min_distance[i] = distance
                        class_min_distance[i]= name_class
                        break
                        
    predict_class = max(set(class_min_distance), key=class_min_distance.count)
    return predict_class

In [64]:
#Validation
def validate(sampling_count):
    right = 0.0;
    total = 0.0;
    accuracy = 0.0;
    class_names = get_class_names(validation_folder_path)
    for class_name in class_names:
        validate_img_paths = glob.glob(validation_folder_path + class_name + '/' +img_type)
        for validate_img_path in validate_img_paths:
            img_validate = load_img(validate_img_path)
            predict_class = knn(img_validate, sampling_count, neighbor)
            class_image = class_name
            if(predict_class == class_image):
                right+=1
            total +=1
    accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) +"%")

In [65]:
#Testing

def testing_knn(feature, neighbor):
    min_distance = []
    class_min_distance = []
    k = 0
    distance = 0.0
    for name_class in model:
        for img_train in model[name_class]:
            distance = euclidian_distance(feature, img_train)
            
            if(k<neighbor):
                min_distance.append(distance)
                class_min_distance.append(name_class)
                k+=1
            else:
                for i in range (k):
                    if(distance<min_distance[i]):
                        min_distance[i] = distance
                        class_min_distance[i]= name_class
                        break
                        
    predict_class = max(set(class_min_distance), key=class_min_distance.count)
    return predict_class

def testing_split(sampling_count):
    right = 0.0;
    total = 0.0;
    accuracy = 0.0;
    class_names = get_class_names(validation_folder_path)
    for class_name in class_names:
        for test_img in model_test[class_name]:
            predict_class = testing_knn(test_img, neighbor)
            class_image = class_name
            if(predict_class == class_image):
                right+=1
            total +=1
    accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) +"%")
    
    
def testing(sampling_count):
    testing_img_paths = glob.glob(test_folder_path + '/' + img_type)
    for testing_img_path in testing_img_paths:
        imgs = load_img(testing_img_path)
        predict_class = knn(imgs, sampling_count, neighbor)
        print("Categorized as " + predict_class)
        plt.imshow(Image.open(testing_img_path))    

In [70]:
#Training Testing

learning_rate = 0.8
sampling_count = 128
img_type = '*.jpg'
root_path = './'
training_folder_path = root_path + 'glcm/training/'
validation_folder_path = root_path + 'glcm/validation/'
test_folder_path = root_path + 'glcm/test/'

class_names = get_class_names(training_folder_path)

for i in range(1):
    for class_name in class_names:
        training_img_paths = glob.glob(training_folder_path + class_name + '/' + img_type)
        for training_img_path in training_img_paths:
            training_img = load_img(training_img_path)
            training_img_class_name = class_name
            feature = feature_extract(training_img, sampling_count, training_img_class_name)
            make_a_model(class_name, feature, learning_rate)
        
#     testing_split(sampling_count)
    models = to_dataframe(model)
    models_test = to_dataframe(model_test)



time_modelling_stop = timeit.default_timer()
print('Time elapsed to make this model is ' + str(time_modelling_stop - time_modelling_start) + ' seconds.')

Time elapsed to make this model is 18.59775590000004 seconds.


In [146]:
total_tulis = models.loc[models['class'] == 'tulis']
total_cap = models.loc[models['class'] == 'cap']
mean= []
variance = []
for a in ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity']:
    mean.append(models[a].mean())
    variance.append(models[a].var())

contrast         0.498655
homogeneity      0.004492
energy           0.011133
correlation      0.046286
ASM              0.007225
dissimilarity    0.047001
dtype: float64

In [144]:
mean, variance

([2.034089707872138,
  0.6821107182771711,
  0.40702823099527624,
  0.3769980496921079,
  0.18863062077316067,
  0.8497529973878559],
 [2.30667164868116,
  0.015147573171397869,
  0.023013564921964335,
  0.058339783371230046,
  0.02022692028959421,
  0.18180489548007436])

In [111]:
a = (120-90)/(2*25)
b = 1/(np.sqrt(2*3.14*25))

In [112]:
b = b * np.exp(a)
b

0.14542091165335422

In [113]:
30/50

0.6